In [31]:
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import time
import matplotlib.pyplot as plt
import numpy as np

In [32]:
os.listdir("../../data/static_files/")

['LargeCapCD.xlsx',
 'LargeCapComms.xlsx',
 'LargeCapCS.xlsx',
 'LargeCapEner.xlsx',
 'LargeCapFina.xlsx',
 'LargeCapHC.xlsx',
 'LargeCapInd.xlsx',
 'LargeCapIT.xlsx',
 'LargeCapMat.xlsx',
 'LargeCapRE.xlsx',
 'LargeCapUtil.xlsx',
 'nasdaq_nyse.xlsx']

In [33]:
load_dotenv("../../.env",override=True)
consumerKey = os.environ.get("CONSUMER_KEY")


In [34]:
pd.set_option("display.max_columns",500)
pd.set_option("display.max_rows",10)

In [35]:
def request_fundamentals(Tickers: list, consumerKey: str, how: str = 'DF') -> pd.DataFrame:
    """_summary_

    Args:
        Tickers (list)
        consumerKey (str)

    Returns:
        pd.DataFrame
    """
    symbolList = ",".join([i for i in Tickers])
    endpoint = f'''https://api.tdameritrade.com/v1/instruments?
        &symbol={symbolList}&projection=fundamental'''
    page = requests.get(url=endpoint, 
        params={'apikey' : consumerKey})
    content = json.loads(page.content)

    allDict = {}
    for k in list(content.keys()):
        allDict[k] = content[k]['fundamental']
    fundamentals = pd.DataFrame(allDict).T
    
    if how=='DF':
        return fundamentals
    else:
        return allDict

def request_quotes(consumerKey: str, Tickers: list) -> pd.DataFrame:
    """_summary_

    Args:
        consumerKey (str): _description_
        Tickers (list): _description_

    Returns:
        pd.DataFrame: _description_
    """
    
    symbolList = ",".join([i for i in Tickers])
    endpoint = f"https://api.tdameritrade.com/v1/marketdata/quotes?symbol={symbolList}"
    page = requests.get(url=endpoint, 
                params={'apikey' : consumerKey})
    content = json.loads(page.content)
    content = pd.DataFrame(content).T
    return content

In [36]:
suffixes = ['CD','Comms','CS','Ener','Fina','HC','Ind','IT','Mat','RE','Util']

In [37]:
Industry = suffixes[7]
symbols = pd.read_excel(f"../../data/static_files/LargeCap{Industry}.xlsx",header=1)
symbols['Symbol'].fillna("-",inplace=True)
Tickers = list(symbols[(~symbols['Symbol'].astype(str).str.contains('-'))]['Symbol'])


c:\Users\jliv\Desktop\research\python_research\.venv\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [38]:

FDF = request_fundamentals(Tickers=Tickers,consumerKey=consumerKey)
Quotes = request_quotes(Tickers=Tickers,consumerKey=consumerKey)

In [39]:
dat = Quotes[['symbol','description','lastPrice',
    'volatility','peRatio','divAmount','divYield',
    ]].merge(FDF[['symbol']+[
        i for i in FDF.columns if i not in Quotes.columns
    ]],on='symbol',how='inner')

dat=pd.DataFrame(np.where(dat==0,np.nan,dat),columns = dat.columns)

In [40]:
rankdat = dat.copy()

In [41]:
metricsFamilies = {
    'debt': {'columns': ['totalDebtToCapital','ltDebtToEquity','totalDebtToEquity'],
            'ascending':True},
    'ratio': {'columns': ['quickRatio','currentRatio','interestCoverage'],
            'ascending':False},
    'change': {'columns': ['epsChangePercentTTM',
       'epsChangeYear', 'epsChange', 'revChangeYear', 'revChangeTTM',
       'revChangeIn'],
            'ascending':False},
    'profit': {'columns': ['epsTTM','grossMarginTTM', 'grossMarginMRQ', 'netProfitMarginTTM',
       'netProfitMarginMRQ', 'operatingMarginTTM', 'operatingMarginMRQ'],
            'ascending':False},
    'value': {'columns': ['peRatio','pegRatio', 'pbRatio', 'prRatio',
       'pcfRatio'],
            'ascending':True},
    'return': {'columns':  ['returnOnEquity', 'returnOnAssets', 'returnOnInvestment'],
            'ascending':False},
}

In [42]:
def convert_to_ranking(rankdat: pd.DataFrame, 
    metricsFamilies: dict[str,dict]) -> pd.DataFrame:
    """_summary_

    Args:
        rankdat (pd.DataFrame): _description_
        metricsFamilies (dict[str,dict]): _description_

    Returns:
        pd.DataFrame: _description_
    """
    families = list(metricsFamilies.keys())
    
    for f in families:
        columns = metricsFamilies[f]['columns']
        ascending= metricsFamilies[f]['ascending']
        for v in columns:
            rankdat[v] = rankdat[v].rank(ascending=ascending)

        rankdat[f'{f}Known']=np.sum(np.where(rankdat[columns].isna(),0,1),axis=1)

        rankdat[columns]=rankdat[columns]/np.nanmax(rankdat[columns],axis=0)

    return rankdat


In [43]:
rankdat = convert_to_ranking(rankdat,metricsFamilies)

C:\Users\jliv\AppData\Local\Temp\ipykernel_20252\933486261.py:22: RuntimeWarning: All-NaN axis encountered
  rankdat[columns]=rankdat[columns]/np.nanmax(rankdat[columns],axis=0)


In [44]:
rankdat = convert_to_ranking(rankdat,metricsFamilies)
rankedCols = []
for k in metricsFamilies.keys():
    rankedCols=rankedCols+metricsFamilies[k]['columns']
    
knownCols = [i for i in rankdat.columns if 'Known' in i]
rankdat['metricsPositive'] = np.sum(rankdat[knownCols],axis=1)

rankdat[rankedCols]=rankdat[rankedCols].copy().fillna(1)

rankdat['meanRank']=np.mean(rankdat[rankedCols],axis=1)


C:\Users\jliv\AppData\Local\Temp\ipykernel_20252\933486261.py:22: RuntimeWarning: All-NaN axis encountered
  rankdat[columns]=rankdat[columns]/np.nanmax(rankdat[columns],axis=0)


In [45]:
rankdat

,symbol,description,lastPrice,volatility,peRatio,divAmount,divYield,high52,low52,dividendAmount,dividendYield,dividendDate,pegRatio,pbRatio,prRatio,pcfRatio,grossMarginTTM,grossMarginMRQ,netProfitMarginTTM,netProfitMarginMRQ,operatingMarginTTM,operatingMarginMRQ,returnOnEquity,returnOnAssets,returnOnInvestment,quickRatio,currentRatio,interestCoverage,totalDebtToCapital,ltDebtToEquity,totalDebtToEquity,epsTTM,epsChangePercentTTM,epsChangeYear,epsChange,revChangeYear,revChangeTTM,revChangeIn,sharesOutstanding,marketCapFloat,marketCap,bookValuePerShare,shortIntToFloat,shortIntDayToCover,divGrowthRate3Year,dividendPayAmount,dividendPayDate,beta,vol1DayAvg,vol10DayAvg,vol3MonthAvg,debtKnown,ratioKnown,changeKnown,profitKnown,valueKnown,returnKnown,metricsPositive,meanRank
0,AAPL,Apple Inc. - Common Stock,147.96,0.0266,0.423077,0.92,0.62,182.94,129.04,0.92,0.62,2022-08-05 00:00:00.000,0.684783,0.961290,0.597633,0.565217,0.294479,0.314815,0.792308,0.763780,0.834586,0.767442,0.958333,0.968992,0.968992,0.150,0.084337,1.000000,0.802721,0.832061,0.879699,0.700000,0.326923,1.000000,1.0,1.0,0.318182,1.000000,16070752000.0,16059.34,2366740.0,NaN,NaN,NaN,NaN,0.23,2022-08-11 00:00:00.000,1.24754,82140490.0,82140485.0,1740998110.0,3,2,2,7,5,3,22,0.703320
1,ACN,Accenture plc Class A Ordinary Shares (Ireland),273.33,0.0843,0.476923,4.48,1.66,417.37,242.95,4.48,1.66,2022-10-12 00:00:00.000,0.706522,0.645161,0.266272,0.557971,0.122699,0.111111,0.323077,0.393701,0.383459,0.379845,0.733333,0.767442,0.837209,1.000,0.307229,1.000000,0.006803,0.007634,0.007519,0.923077,0.278846,0.329412,1.0,1.0,0.545455,1.000000,664783164.0,663.4805,179205.6,19.99096,NaN,NaN,NaN,1.12,2022-11-15 00:00:00.000,1.21025,2288520.0,2288523.0,43511990.0,3,1,3,7,5,3,22,0.522618
2,ADBE,Adobe Inc. - Common Stock,311.6797,0.0672,0.592308,NaN,NaN,699.54,274.73,NaN,NaN,,1.000000,0.735484,0.710059,0.644928,0.975460,0.987654,0.853846,0.818898,0.894737,0.844961,0.741667,0.837209,0.821705,1.000,0.240964,0.848485,0.238095,0.267176,0.263158,0.915385,1.000000,1.000000,1.0,1.0,0.389610,0.038168,464900000.0,462.9087,142431.4,15.70408,NaN,NaN,NaN,NaN,,1.24186,4034410.0,4034414.0,83040740.0,3,2,2,7,4,3,21,0.728147
3,ADI,"Analog Devices, Inc. - Common Stock",145.345,0.074,0.707692,3.04,2.07,191.95,133.48,3.04,2.07,2022-08-29 00:00:00.000,1.000000,0.174194,0.633136,0.463768,0.496933,0.586420,0.584615,0.787402,0.511278,0.790698,0.125000,0.271318,0.193798,0.475,0.596386,0.717172,0.149660,0.190840,0.165414,0.515385,1.000000,0.117647,1.0,1.0,0.967532,0.343511,514341531.0,509.603,75397.33,33.96363,NaN,NaN,NaN,0.76,2022-09-08 00:00:00.000,1.14291,3885550.0,3885547.0,73356280.0,3,3,3,7,4,3,23,0.539437
4,ADP,"Automatic Data Processing, Inc. - Common Stock",236.41,0.0941,0.638462,4.16,1.79,261.59,192.26,4.16,1.79,2022-09-08 00:00:00.000,0.782609,0.935484,0.573964,0.695652,0.282209,0.296296,0.600000,0.551181,0.624060,0.558140,0.883333,0.286822,0.891473,1.000,0.120482,0.888889,0.646259,0.748092,0.714286,0.769231,0.240385,0.317647,1.0,1.0,0.207792,1.000000,415291672.0,414.824,96654.98,NaN,NaN,NaN,NaN,1.04,2022-10-01 00:00:00.000,0.82482,1663140.0,1663135.0,35527200.0,3,2,3,7,5,3,23,0.638991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,WU,Western Union Company (The) Common Stock,13.775,0.0364,0.053846,0.94,6.9,20.4,13.05,0.94,6.9,2022-09-15 00:00:00.000,0.391304,0.780645,0.094675,0.086957,0.306748,0.308642,0.615385,0.614173,0.729323,0.658915,0.975000,0.503876,0.449612,1.000,1.000000,0.464646,0.884354,0.977099,0.977444,0.330769,0.211538,1.000000,1.0,1.0,1.000000,1.000000,385754381.0,382.9503,5257.832,NaN,NaN,NaN,NaN,0.235,2022-09-30 00:00:00.000,0.91578,5497980.0,5497984.0,93226900.0,3,1,1,7,5,3,20,0.644998
165,XM,Qualtrics International Inc. - Class A Common ...,9.75,0.1026,1.000000,NaN,NaN,48.0,9.32,NaN,NaN,,1.0

In [47]:
out=rankdat[['symbol','description','meanRank','metricsPositive']].merge(
    dat[['symbol']+rankedCols],
    on=['symbol'],
    how='left'
)

In [48]:
np.round(out.sort_values(by='meanRank').head(10),2)

,symbol,description,meanRank,metricsPositive,totalDebtToCapital,ltDebtToEquity,totalDebtToEquity,quickRatio,currentRatio,interestCoverage,epsChangePercentTTM,epsChangeYear,epsChange,revChangeYear,revChangeTTM,revChangeIn,epsTTM,grossMarginTTM,grossMarginMRQ,netProfitMarginTTM,netProfitMarginMRQ,operatingMarginTTM,operatingMarginMRQ,peRatio,pegRatio,pbRatio,prRatio,pcfRatio,returnOnEquity,returnOnAssets,returnOnInvestment
104,NOK,Nokia Corporation Sponsored American Depositar...,0.33,25,22.33425,26.64973,28.9205,1.28291,1.53349,69.0,196.7038,20.92486,NaN,NaN,6.87078,6.26596,0.29621,40.0779,40.1378,7.28765,6.64958,9.10957,8.28393,14.3477,0.072941,1.2263,1.01941,8.51029,9.59502,4.25043,6.2093
71,HPE,Hewlett Packard Enterprise Company Common Stock,0.34,25,40.18064,44.3307,67.34268,0.65576,0.91652,97.5,259.0838,5.52275,NaN,NaN,1.23381,3.54536,2.79536,33.35359,34.46986,13.31356,5.88405,13.49941,6.27248,4.8187,0.018599,0.84175,0.61946,2.78289,19.79751,6.52867,10.19626
58,FLEX,Flex Ltd. - Ordinary Shares,0.35,24,49.39394,76.48497,99.6089,0.68197,1.3069,6.55814,26.95982,NaN,NaN,NaN,6.84628,7.23982,1.92725,7.46506,7.28188,3.43489,2.65414,4.48125,3.70219,9.319,0.345662,2.00867,0.30286,5.77257,24.18739,5.12608,11.30583
51,ERIC,Ericsson - American Depositary Shares each rep...,0.36,24,23.61801,24.67624,30.52227,0.81599,1.16552,13.76877,12.82702,NaN,NaN,NaN,11.41769,8.925,0.61037,42.36896,41.43151,8.97944,7.87772,12.07795,10.45708,9.0765,0.707609,1.4889,0.79514,6.36001,19.27182,7.07941,12.21709
23,CAJ,"Canon, Inc. American Depositary Shares",0.37,23,8.56219,5.75574,10.05258,1.16696,1.84402,NaN,22.0115,NaN,NaN,NaN,6.93229,13.58378,1.38925,45.93062,46.4596,6.2862,6.30407,8.4185,9.85934,16.2173,0.736766,1.0936,1.1864,9.54284,7.28437,4.65611,6.62955
135,SNX,TD SYNNEX Corporation Common Stock,0.37,22,34.72539,49.95394,53.19893,0.6357,1.24804,5.52489,NaN,NaN,NaN,NaN,178.9433,0.56513,5.70912,6.13162,5.96526,0.89052,0.96922,1.46187,1.57289,15.1757,NaN,1.05868,0.13416,8.18818,10.80366,2.7754,5.58997
14,ARW,"Arrow Electronics, Inc. Common Stock",0.38,25,39.21414,53.48765,65.21038,1.04575,1.48865,11.61597,80.26684,71.75842,NaN,NaN,10.50877,4.26176,20.0995,13.07659,13.06664,3.88074,3.92666,5.43487,5.63193,4.9131,0.061209,1.19457,0.17572,3.9822,26.58289,7.61913,16.94952
56,FIS,"Fidelity National Information Services, Inc. C...",0.38,23,28.42799,25.27631,39.87658,NaN,0.68946,25.1875,864.3723,NaN,NaN,NaN,8.92438,6.50057,1.37722,38.11675,39.93009,5.92773,7.52891,7.81098,10.05647,56.5124,0.06538,1.01752,3.28832,9.59428,1.78297,1.05368,1.26826
160,WDC,Western Digital Corporation - Common Stock,0.39,24,36.49119,57.45847,57.45847,1.11037,1.80504,9.80822,80.91598,NaN,NaN,NaN,11.05661,3.3554,4.7525,33.40073,34.89399,7.9817,6.64753,12.72282,12.41166,7.3351,0.090651,0.89853,0.58905,4.55741,13.07645,5.72617,7.09488
83,JNPR,"Juniper Networks, Inc. Common Stock",0.40,25,27.67555,38.26582,38.26582,1.37294,1.57547,10.93939,91.04878,84.18865,NaN,NaN,6.92349,8.68002,1.18242,56.55333,54.67076,7.94479,8.97133,9.86705,12.0983,24.323,0.267143,2.16883,1.88333,14.97941,9.1017,4.45167,5.61405


In [49]:
for i in out.columns:
    try:
        out[i]=out[i].astype(float)
    except:
        pass

In [50]:
np.round(out,2)

,symbol,description,meanRank,metricsPositive,totalDebtToCapital,ltDebtToEquity,totalDebtToEquity,quickRatio,currentRatio,interestCoverage,epsChangePercentTTM,epsChangeYear,epsChange,revChangeYear,revChangeTTM,revChangeIn,epsTTM,grossMarginTTM,grossMarginMRQ,netProfitMarginTTM,netProfitMarginMRQ,operatingMarginTTM,operatingMarginMRQ,peRatio,pegRatio,pbRatio,prRatio,pcfRatio,returnOnEquity,returnOnAssets,returnOnInvestment
0,AAPL,Apple Inc. - Common Stock,0.70,22.0,67.32,162.98,205.98,0.82,0.86,NaN,18.52,NaN,NaN,NaN,11.63,NaN,6.05,43.31,43.26,25.71,23.44,30.53,27.82,24.33,1.31,40.79,6.11,21.35,162.82,29.91,46.50
1,ACN,Accenture plc Class A Ordinary Shares (Ireland),0.52,22.0,0.24,0.21,0.25,NaN,1.23,NaN,17.02,18.42,NaN,NaN,21.89,NaN,10.71,31.99,32.07,11.35,10.97,15.05,14.71,25.16,1.48,7.70,2.91,20.95,33.04,15.46,24.96
2,ADBE,Adobe Inc. - Common Stock,0.73,21.0,22.31,25.23,28.71,NaN,1.14,43.65,NaN,NaN,NaN,NaN,13.86,1.07,10.14,87.76,87.68,28.00,25.63,35.45,33.48,30.21,NaN,9.95,8.29,25.17,33.44,18.20,24.52
3,ADI,"Analog Devices, Inc. - Common Stock",0.54,23.0,14.58,17.07,17.07,1.43,1.92,22.74,NaN,6.30,NaN,NaN,70.73,4.64,3.61,58.63,65.70,17.00,24.08,18.55,28.72,40.60,NaN,2.06,6.79,18.43,7.72,5.25,5.66
4,ADP,"Automatic Data Processing, Inc. - Common Stock",0.64,23.0,48.09,92.61,92.65,NaN,0.99,59.20,15.38,18.25,NaN,NaN,9.95,NaN,7.00,42.81,42.35,17.87,15.15,22.91,19.60,33.23,2.16,30.02,5.86,27.90,66.30,5.27,31.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,WU,Western Union Company (The) Common Stock,0.64,20.0,85.74,543.23,601.23,NaN,NaN,11.52,14.10,NaN,NaN,NaN,NaN,NaN,2.23,43.61,42.63,18.27,17.04,26.48,23.19,6.10,0.43,11.74,1.08,4.87,241.98,10.01,12.16
165,XM,Qualtrics International Inc. - Class A Common ...,0.85,8.0,NaN,NaN,NaN,NaN,1.34,NaN,NaN,6.54,NaN,NaN,43.08,6.17,NaN,71.68,70.69,NaN,NaN,NaN,NaN,NaN,NaN,3.22,4.84,NaN,NaN,NaN,NaN
166,ZBRA,Zebra Technologies Corporation - Class A Commo...,0.58,20.0,46.23,80.26,85.99,0.56,0.85,40.50,NaN,NaN,NaN,NaN,12.37,2.51,9.20,45.25,45.91,8.56,NaN,9.08,NaN,28.60,NaN,5.44,2.35,19.49,19.48,7.66,10.67
167,ZM,"Zoom Video Communications, Inc. - Class A Comm...",0.67,18.0,NaN,NaN,NaN,NaN,3.44,3.51,NaN,NaN,NaN,NaN,18.09,2.39,3.23,75.24,75.11,23.06,4.16,19.82,11.07,25.04,NaN,4.03,5.61,22.95,18.60,13.61,18.00


In [ ]:


request_fundamentals(Tickers=['TWLO'],consumerKey=consumerKey,how='Dict')

In [ ]:
symbolList = ",".join([i for i in ['INTC']])
endpoint = f'''https://api.tdameritrade.com/v1/instruments?
    &symbol={symbolList}&projection=fundamental'''
page = requests.get(url=endpoint, 
    params={'apikey' : consumerKey})

page.content

In [ ]:
endpoint